In [54]:
import json

with open('../data/train_dataset.json') as f:
    data = json.load(f)
    
print(len(data))
    
with open('../data/dev_dataset.json') as f:
    data += json.load(f)
    
print(len(data))

168300
177100


In [15]:
data[130000]

{'tokens': ['марка', 'лыж', 'и', 'ботинок', '—', 'fischer'],
 'token_labels': ['O', 'O', 'O', 'O', 'O', 'B-CORP'],
 'lang': 'RU-Russian',
 'id': 'b894d113-5781-4557-ac92-4df41c3676a0',
 'domain': 'train'}

In [20]:
data[-2100]

{'tokens': ['71', '134аэн', '—', 'трамвайный', 'вагон', 'лм-99аэн'],
 'token_labels': ['O', 'O', 'O', 'O', 'O', 'B-PROD'],
 'lang': 'RU-Russian',
 'id': 'f5fc6956-ca44-44be-8760-29497c58bb8d',
 'domain': 'dev'}

* * *

Для начала посчитаю следующие статистики:

* количество типов разных сущностей
* средняя длина сущностей разного типа
* топ 10 сущностей каждого типа 
* есть ли вложенность / пересечения между сущностями

In [33]:
from collections import Counter

def count_entities(sample):
    collapsed_entities = []
    saw_begin_before = False

    for label in sample['token_labels']:
        if label == 'O':
            saw_begin_before = False
        elif label[0] == 'B':
            saw_begin_before = True
            collapsed_entities.append(label[2:])
        elif label[0] == 'I':
            if saw_begin_before:
                continue
            else:
                raise ValueError("Found I-label without B-label before")
    
    c = dict(Counter(collapsed_entities))
    c['domain'] = sample['domain']
    c['lang'] = sample['lang']
    return c

In [46]:
import pandas as pd
entity_counts = pd.DataFrame(
    [count_entities(sample) for sample in data]
).fillna(0)

entity_counts.head(10)

,CORP,domain,lang,GRP,PER,CW,PROD,LOC
0,1.0,train,BN-Bangla,0.0,0.0,0.0,0.0,0.0
1,0.0,train,BN-Bangla,1.0,0.0,0.0,0.0,0.0
2,0.0,train,BN-Bangla,0.0,1.0,0.0,0.0,0.0
3,1.0,train,BN-Bangla,0.0,0.0,0.0,0.0,0.0
4,0.0,train,BN-Bangla,1.0,0.0,0.0,0.0,0.0
5,0.0,train,BN-Bangla,0.0,1.0,0.0,0.0,0.0
6,0.0,train,BN-Bangla,0.0,1.0,0.0,0.0,0.0
7,0.0,train,BN-Bangla,0.0,1.0,0.0,0.0,0.0
8,0.0,train,BN-Bangla,0.0,0.0,1.0,0.0,0.0
9,0.0,train,BN-Bangla,0.0,0.0,1.0,0.0,0.0


In [47]:
entity_names = [c for c in entity_counts.columns if c not in ['domain', 'lang']]

Cписок сущностей:

In [49]:
entity_names

['CORP', 'GRP', 'PER', 'CW', 'PROD', 'LOC']

Доля каждой из сущностей во всем датасете:

In [48]:
entity_counts[entity_names].mean()

CORP    0.195528
GRP     0.194715
PER     0.261406
CW      0.228052
PROD    0.208741
LOC     0.321637
dtype: float64

В разрезе train/dev

In [50]:
entity_counts.groupby(['domain'])[entity_names].mean()

,CORP,GRP,PER,CW,PROD,LOC
domain,,,,,,
dev,0.197500,0.186136,0.266136,0.228977,0.210000,0.333182
domain_adapt,0.060909,0.050909,0.195455,0.152727,0.048182,0.531818
train,0.196310,0.196112,0.261591,0.228499,0.209731,0.319647


В основном, пропорции сохраняются. Но что такое "domain_adapt"?

In [52]:
(entity_counts['domain'] == 'domain_adapt').sum()

1100

In [53]:
entity_counts.index[entity_counts['domain'] == 'domain_adapt']

Int64Index([ 15200,  15201,  15202,  15203,  15204,  15205,  15206,  15207,
             15208,  15209,
            ...
            168290, 168291, 168292, 168293, 168294, 168295, 168296, 168297,
            168298, 168299],
           dtype='int64', length=1100)

Эта штука встречается только в train выборке. Дальше ее считаю просто трейном. Сейчас внесу это в датафрейм и пересчитаю средние. 

In [55]:
entity_counts.loc[entity_counts['domain'] == 'domain_adapt', 'domain'] = 'train'

In [56]:
entity_counts.groupby(['domain'])[entity_names].mean()

,CORP,GRP,PER,CW,PROD,LOC
domain,,,,,,
dev,0.197500,0.186136,0.266136,0.228977,0.210000,0.333182
train,0.195425,0.195163,0.261159,0.228004,0.208675,0.321034


Думаю, можно считать, что пропорции по количеству сущностей сохранены. 



Теперь аналогично в разрезе еще и языков

In [61]:
entity_counts.groupby(['lang', 'domain'])[entity_names].agg(['count', 'sum', 'mean'])

CORP                      GRP                      PER  \
                   count     sum      mean  count     sum      mean  count   
lang       domain                                                            
BN-Bangla  dev       800   127.0  0.158750    800   118.0  0.147500    800   
           train   15300  2598.0  0.169804  15300  2405.0  0.157190  15300   
DE-German  dev       800   165.0  0.206250    800   160.0  0.200000    800   
           train   15300  3083.0  0.201503  15300  3509.0  0.229346  15300   
EN-English dev       800   193.0  0.241250    800   190.0  0.237500    800   
           train   15300  3111.0  0.203333  15300  3571.0  0.233399  15300   
ES-Spanish dev       800   141.0  0.176250    800   168.0  0.210000    800   
           train   15300  2898.0  0.189412  15300  3226.0  0.210850  15300   
FA-Farsi   dev       800   160.0  0.200000    800   164.0  0.205000    800   
           train   15300  2991.0  0.195490  15300  3199.0  0.209085  15300   
HI-Hindi   dev       800   134.0  0.167500    800   148.0  0.185000    800   
           train   15300  2700.0  0.176471  15300  2843.0  0.185817  15300   
KO-Korean  dev       800   156.0  0.195000    800   183.0  0.228750    800   
           train   15300  3313.0  0.216536  15300  3530.0  0.230719  15300   
NL-Dutch   dev       800   163.0  0.203750    800   163.0  0.203750    800   
           train   15300  2813.0  0.183856  15300  3306.0  0.216078  15300   
RU-Russian dev       800   159.0  0.198750    800   151.0  0.188750    800   
           train   15300  2817.0  0.184118  15300  2976.0  0.194510  15300   
TR-Turkish dev       800   148.0  0.185000    800   167.0  0.208750    800   
           train   15300  2761.0  0.180458  15300  3568.0  0.233203  15300   
ZH-Chinese dev       800   192.0  0.240000    800    26.0  0.032500    800   
           train   15300  3805.0  0.248693  15300   713.0  0.046601  15300   

                                        CW                     PROD          \
                      sum      mean  count     sum      mean  count     sum   
lang       domain                                                             
BN-Bangla  dev      144.0  0.180000    800   120.0  0.150000    800   190.0   
           train   2606.0  0.170327  15300  2157.0  0.140980  15300  3188.0   
DE-German  dev      296.0  0.370000    800   189.0  0.236250    800   133.0   
           train   5288.0  0.345621  15300  3507.0  0.229216  15300  2961.0   
EN-English dev      290.0  0.362500    800   176.0  0.220000    800   147.0   
           train   5397.0  0.352745  15300  3752.0  0.245229  15300  2923.0   
ES-Spanish dev      247.0  0.308750    800   192.0  0.240000    800   154.0   
           train   4706.0  0.307582  15300  3690.0  0.241176  15300  3040.0   
FA-Farsi   dev      201.0  0.251250    800   207.0  0.258750    800   157.0   
           train   4272.0  0.279216  15300  3694.0  0.241438  15300  2955.0   
HI-Hindi   dev      133.0  0.166250    800   113.0  0.141250    800   169.0   
           train   2418.0  0.158039  15300  2304.0  0.150588  15300  3077.0   
KO-Korean  dev      267.0  0.333750    800   196.0  0.245000    800   177.0   
           train   4536.0  0.296471  15300  3883.0  0.253791  15300  3082.0   
NL-Dutch   dev      212.0  0.265000    800   182.0  0.227500    800   138.0   
           train   4408.0  0.288105  15300  3340.0  0.218301  15300  2935.0   
RU-Russian dev      192.0  0.240000    800   168.0  0.210000    800   151.0   
           train   3683.0  0.240719  15300  3224.0  0.210719  15300  2921.0   
TR-Turkish dev      231.0  0.288750    800   190.0  0.237500    800   158.0   
           train   4414.0  0.288497  15300  3574.0  0.233595  15300  3184.0   
ZH-Chinese dev      129.0  0.161250    800   282.0  0.352500    800   274.0   
           train   2225.0  0.145425  15300  5248.0  0.343007  15300  4854.0   

                               LOC                    
                       mean  count    

Вроде разница не особо большая. Самое большое, что я нашел -- это 0.6 для турецкого и сущности LOC. 

Интересно, что во всех языках строго поровну текстов с каждой из сущностей. А вот количество сущностей в тексте может отличаться, похоже, что именно это и даёт вышеуказанную разницу в доле числа сущностей для разных языков. Если я прав, то при подсчете не числа сущностей, а числа текстов, в которых сущность есть хотя бы одна, мы получим ровные совпадения. 

In [67]:
entity_counts[entity_names] = (entity_counts[entity_names] > 0).astype(int)

In [70]:
entity_counts.groupby(['domain']).mean()

,CORP,GRP,PER,CW,PROD,LOC
domain,,,,,,
dev,0.182614,0.166023,0.215341,0.20250,0.188068,0.215682
train,0.180392,0.173987,0.212299,0.20082,0.184302,0.213553


нет, ровнее не стало. 

In [71]:
entity_counts.groupby(['lang', 'domain']).mean()

CORP       GRP       PER        CW      PROD       LOC
lang       domain                                                            
BN-Bangla  dev     0.158750  0.147500  0.180000  0.150000  0.237500  0.126250
           train   0.169739  0.157190  0.170327  0.140980  0.208366  0.153399
DE-German  dev     0.191250  0.175000  0.280000  0.203750  0.145000  0.236250
           train   0.182549  0.199020  0.256405  0.199281  0.168954  0.202810
EN-English dev     0.216250  0.205000  0.261250  0.183750  0.161250  0.190000
           train   0.183137  0.204052  0.258562  0.208497  0.165163  0.208039
ES-Spanish dev     0.163750  0.188750  0.238750  0.212500  0.166250  0.243750
           train   0.170196  0.192026  0.243072  0.212941  0.169608  0.215294
FA-Farsi   dev     0.176250  0.181250  0.207500  0.211250  0.175000  0.232500
           train   0.178497  0.180719  0.227451  0.198954  0.170261  0.217712
HI-Hindi   dev     0.166250  0.185000  0.165000  0.140000  0.210000  0.151250
           train   0.175686  0.184510  0.156144  0.147582  0.199150  0.158627
KO-Korean  dev     0.180000  0.188750  0.256250  0.222500  0.183750  0.222500
           train   0.194118  0.192549  0.231765  0.225359  0.163725  0.238758
NL-Dutch   dev     0.188750  0.181250  0.217500  0.206250  0.155000  0.241250
           train   0.167974  0.190065  0.229935  0.191307  0.168824  0.230850
RU-Russian dev     0.183750  0.178750  0.217500  0.193750  0.171250  0.208750
           train   0.173007  0.181961  0.218366  0.190458  0.166340  0.207908
TR-Turkish dev     0.167500  0.163750  0.230000  0.202500  0.173750  0.240000
           train   0.165294  0.191307  0.229804  0.202026  0.172941  0.219673
ZH-Chinese dev     0.216250  0.031250  0.115000  0.301250  0.290000  0.280000
           train   0.224118  0.040458  0.113464  0.291634  0.273987  0.296013

Впрочем, так разника получается намного меньше. 

# Посмотрим теперь на сами сущности

In [ ]:
def extract_entities(sample):
    extracted_entities = {}

    buffer = []
    for token, label in zip(sample['tokens'], sample['token_labels']):
        if label == 'O':
            if buffer:
                key = 
            
        elif label[0] == 'B':
            
            buffer.append((token, label))
        elif label[0] == 'I':
            buffer.append((token, label))
            
    
    c = dict(Counter(collapsed_entities))
    c['domain'] = sample['domain']
    c['lang'] = sample['lang']
    return c

In [72]:
data[6]

{'tokens': ['এটি',
  'জিয়ান',
  'লুইগি',
  'রন্ডি',
  'পরিচালিত',
  'শেষ',
  'সংস্করণ',
  'ছিল।'],
 'token_labels': ['O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O'],
 'lang': 'BN-Bangla',
 'id': '51a7a62a-1a30-413b-bc09-fec3f7110821',
 'domain': 'train'}